In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import pandas as pd

In [2]:
# 将JSON文件转换为CSV文件
df = pd.read_json('huanhuan.json')
ds = Dataset.from_pandas(df)

In [3]:
ds[:3]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。'],
 'input': ['', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。', '你们俩话太多了，我该和温太医要一剂药，好好治治你们。', '出来走走，也是散心。']}

In [4]:
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/THUCoAI/CharacterGLM-6B", trust_remote_code=True)
tokenizer

ChatGLMTokenizer(name_or_path='/root/autodl-tmp/THUCoAI/CharacterGLM-6B', vocab_size=64794, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'eos_token': '</s>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	64790: AddedToken("[gMASK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	64792: AddedToken("sop", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [5]:
print(tokenizer.encode(ds[0]['instruction']))

[64790, 64792, 35182, 55671, 31123, 34752, 55276, 54740, 32595, 54806, 54538, 54878, 31123, 37963, 35662, 36028, 54695, 54732, 60136, 49127, 31123, 34856, 31781, 33498, 54792, 54792, 40972, 16747]


In [6]:
def build_prompt(self, query, history=None):
        if history is None:
            history = []
        prompt = ""
        for i, (old_query, response) in enumerate(history):
            prompt += "[Round {}]\n\n用户：{}\n\nCharacterGLM-6B：{}\n\n".format(i + 1, old_query, response)
        prompt += "[Round {}]\n\n用户：{}\n\nCharacterGLM-6B：".format(len(history) + 1, query)
        return prompt

In [7]:
demo_token = build_prompt('用户',"现在你要扮演皇帝身边的女人--甄嬛")
demo_token

'[Round 1]\n\n用户：现在你要扮演皇帝身边的女人--甄嬛\n\nCharacterGLM-6B：'

In [8]:
tokenizer.get_command("[gMASK]"), tokenizer._convert_id_to_token(+tokenizer.eos_token_id)

(64790, '')

# 调用api处理数据

In [9]:
instruction = "\n".join([ds[0]["instruction"], ds[0]["input"]]).strip()
instruction

'小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——'

In [10]:
instruction1 = build_prompt("用户",instruction)
response = tokenizer("\n" + ds[0]["output"], add_special_tokens=False)

In [11]:
instruction1

'[Round 1]\n\n用户：小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——\n\nCharacterGLM-6B：'

In [12]:
response

{'input_ids': [30910, 13, 58694, 16747, 38048, 55124, 55213, 54636, 55251, 32792, 55462, 54530, 31155], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'position_ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}

In [13]:
# input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id]
input_ids =response["input_ids"] + [tokenizer.eos_token_id]
answer=tokenizer.decode(input_ids)

In [14]:
answer

'\n嘘——都说许愿说破是不灵的。'

In [15]:
def process_func(example):
    MAX_LENGTH = 512
    input_ids, labels = [], []
    prompt = tokenizer.encode("用户:\n"+"现在你要扮演皇帝身边的女人--甄嬛。", add_special_tokens=False)
    instruction_ = tokenizer.encode("\n".join([example["instruction"], example["input"]]).strip(), add_special_tokens=False,max_length=512)
    instruction = tokenizer.encode(prompt + instruction_)
    response = tokenizer.encode("CharacterGLM-6B:\n:" + example["output"], add_special_tokens=False)
    input_ids = instruction + response + [tokenizer.eos_token_id]
    labels = [tokenizer.pad_token_id] * len(instruction) + response + [tokenizer.eos_token_id]
    pad_len = MAX_LENGTH - len(input_ids)
    # print()
    input_ids += [tokenizer.pad_token_id] * pad_len
    labels += [tokenizer.pad_token_id] * pad_len
    labels = [(l if l != tokenizer.pad_token_id else -100) for l in labels]

    return {
        "input_ids": input_ids,
        "labels": labels
    }

In [16]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/3729 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 3729
})

In [17]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'[gMASK] sop 用户:\n现在你要扮演皇帝身边的女人--甄嬛。 这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。 CharacterGLM-6B:\n:你们俩话太多了，我该和温太医要一剂药，好好治治你们。'

In [18]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'CharacterGLM-6B:\n:你们俩话太多了，我该和温太医要一剂药，好好治治你们。'

# 创建模型

In [19]:
model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/THUCoAI/CharacterGLM-6B", trust_remote_code=True, low_cpu_mem_usage=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


LORA

In [20]:
from peft import LoraConfig, TaskType, get_peft_model
# model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/ZhipuAI/chatglm3-6b", low_cpu_mem_usage=True)
config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules={"query_key_value"}, r=8, lora_alpha=32)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value'}, lora_alpha=32, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [21]:
model = get_peft_model(model, config)

In [22]:
model.print_trainable_parameters()

trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.031217444255383614


In [23]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=None,
    padding=False
)

In [24]:
args = TrainingArguments(
    output_dir="./huanhuan",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=20,
    num_train_epochs=1
)

In [25]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Step,Training Loss
20,5.458500
40,4.000600
60,3.138400
80,2.745400
100,2.460900
120,2.316500
140,2.184600
160,2.212400
180,2.404500
200,2.179000


In [ ]:
model = model.cuda()
ipt = tokenizer("用户：{}\n{}".format("现在你要扮演皇帝身边的女人--甄嬛。你是谁？", "").strip() + "characterGLM-6B:\n", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)